In [221]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

import prettytable as pt

import pydotplus
from sklearn.tree import export_graphviz

from PIL import Image

## 1. Data Input

In [222]:
def load_data():
    path = "./student-mat.csv"
    df = pd.read_csv(path, sep = ';')
    return df

In [223]:
df = load_data()
pd.set_option('display.max_columns', None)
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


## 2. Data Preprocessing

In [224]:
def preprocessing(df):
    for index in df.keys():
        if type(df[index][0]) == type("str") or index == "G3":
            continue
        df[index] = (df[index] - df[index].mean()) / df[index].std()

    df = pd.get_dummies(df)
    df = shuffle(df) #shuffle data
    return df

In [225]:
def get_y2(y):
    less_10 = [i for i in range(10)]
    higher_or_eq_10 = [i for i in range(10, 21, 1)]
    
    y_2 = y
    y_2 = y_2.replace(less_10, 0)
    y_2 = y_2.replace(higher_or_eq_10, 1)
    
    return y_2

In [226]:
def get_y5(y):
    F = [i for i in range(10)]
    D = [10, 11]
    C = [12, 13]
    B = [14, 15]
    A = [i for i in range(16, 21, 1)]
    
    y_5 = y
    y_5 = y_5.replace(F, 0)
    y_5 = y_5.replace(D, 1)
    y_5 = y_5.replace(C, 2)
    y_5 = y_5.replace(B, 3)
    y_5 = y_5.replace(A, 4)
    
    return y_5

In [227]:
df = preprocessing(df)
X = df.drop(columns = ['G3'])
y = df['G3']

y_2 = get_y2(y)
y_5 = get_y5(y)

X = X.to_numpy()
y_2 = y_2.to_numpy()
y_5 = y_5.to_numpy()

## 3. Principal components analysis (PCA)

In [228]:
def getPCA(X):
    pca = PCA(n_components = 20)
    PCA_X = pca.fit(X).transform(X)
    return PCA_X

In [229]:
PCA_X = getPCA(X)

## 4. Model Construction

In [230]:
def decision_tree(X_train, y_train, draw):
    model = DecisionTreeClassifier(criterion = "gini", max_depth = 7, max_leaf_nodes = 10)
    model.fit(X_train, y_train)
    
    if draw == 1:
        labels = ["fail", "pass"]
        dot_data = export_graphviz(model, out_file = None, class_names = labels)
        graph = pydotplus.graph_from_dot_data(dot_data)
        graph.write_pdf("decision_tree.pdf")
    
    return model

In [231]:
def random_forest(X_train, y_train, n):
    model = RandomForestClassifier(n_estimators = n, criterion = "gini", max_depth = 7)
    model.fit(X_train, y_train)
    return model

In [232]:
def KNN(X_train, y_train, n):
    model = KNeighborsClassifier(n_neighbors = n, weights = "distance")
    model.fit(X_train, y_train)
    return model

## 5. Validation

In [233]:
def k_fold(X, y, mode):
    k = 3
    kf = KFold(n_splits = k) #得到一個 k = 3 的 KFold object
    
    size = None
    
    if mode <= 5 or mode >= 12:
        size = 2
    else:
        size = 5
        
    draw = 0
    if mode == 1:
        draw = 1
        
    total_c_m = np.zeros((size, size))
    total_a_s, total_r_s, total_p_s =  0, 0, 0
    #initialize total confusion matrix, total accuracy, total recall, total precision

    for train_index, test_index in kf.split(X):
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index] 
        #先把 training data 跟 testing data 分配好
        
        model = None
        
        if mode == 12:
            model = random_forest(X_train, y_train, 33)
        elif mode == 13:
            model = random_forest(X_train, y_train, 17)
        elif mode == 14:
            model = KNN(X_train, y_train, 11)
        elif mode == 15:
            model = KNN(X_train, y_train, 7)
        elif mode % 6 == 0 or mode % 6 == 1:
            model = decision_tree(X_train, y_train, draw) #拿到model
        elif mode % 6 == 2 or mode % 6 == 3:
            model = random_forest(X_train, y_train, 65)
        elif mode % 6 == 4 or mode % 6 == 5:
            model = KNN(X_train, y_train, 17)
            
        y_pred = model.predict(X_test) #透過此 model 對 testing data 做預測
        c_m = confusion_matrix(y_test, y_pred) #得到 confusion matrix
        a_s = accuracy_score(y_test, y_pred) #得到 accuarcy
        r_s = recall_score(y_test, y_pred, average = None) #得到 recall
        p_s = precision_score(y_test, y_pred, average = None) #得到 precision
        
        total_c_m = total_c_m + c_m  #把 total confusion matrix 加上此次算出來的 confusion matrix
        total_a_s = total_a_s + a_s #把 total accuracy 加上此次算出來的 accuracy
        total_r_s = total_r_s + r_s #把 total recall 加上此次算出來的 recall
        total_p_s = total_p_s + p_s #把 total precision 加上此次算出來的 precision
        
    avg_c_m = np.round(total_c_m / k, 2) #算出平均 confusion matrix
    avg_a_s = round(total_a_s / k, 2) #算出平均 accuracy
    avg_r_s = np.round(total_r_s / k, 2) #算出平均 recall
    avg_p_s = np.round(total_p_s / k, 2) #算出平均 precision

    model_name = None
    with_without = None
    classification = None
    
    if mode == 12:
        model_name = "Random Forest(33 trees)"
    elif mode == 13:
        model_name = "Random Forest(17 trees)"
    elif mode == 14:
        model_name = "KNN(K = 11)"
    elif mode == 15:
        model_name = "KNN(K = 7)"
    elif mode % 6 == 0 or mode % 6 == 1:
        model_name = "Decision Tree"
    elif mode % 6 == 2 or mode % 6 == 3:
        model_name = "Random Forest(65 trees)"
    else:
        model_name = "KNN(K = 17)"
        
    if mode % 2 == 0 or mode >= 12:
        with_without = "without"
    else:
        with_without = "with"

    if mode <= 5 or mode >= 12:
        classification = "Binary Classification"
    else:
        classification = "5-level Classification"
    
    method = "{} {} PCA ({})".format(model_name, with_without, classification)
    
    k_list = None
    if mode <= 5 or mode >= 12:
        k_list = [method, avg_a_s, avg_r_s[0], avg_r_s[1], "/", "/", "/", 
                  avg_p_s[0], avg_p_s[1], "/", "/", "/"]
    
    else:
        k_list = [method, avg_a_s, avg_r_s[0], avg_r_s[1], avg_r_s[2], avg_r_s[3], avg_r_s[4], 
                  avg_p_s[0], avg_p_s[1], avg_p_s[2], avg_p_s[3], avg_p_s[4]]
    #把 method name, 平均 confusion matrix, 平均 accuracy, 平均 recall, 平均 precision 塞進 k_list 中

    c_m_list = [method, avg_c_m]
    return c_m_list, k_list

## 6. Results

In [234]:
d_no_2_c_m, d_no_2_list = k_fold(X, y_2, 0)
d_wi_2_c_m, d_wi_2_list = k_fold(PCA_X, y_2, 1)
r_no_2_c_m, r_no_2_list = k_fold(X, y_2, 2)
r_wi_2_c_m, r_wi_2_list = k_fold(PCA_X, y_2, 3)
K_no_2_c_m, K_no_2_list = k_fold(X, y_2, 4)
K_wi_2_c_m, K_wi_2_list = k_fold(PCA_X, y_2, 5)

d_no_5_c_m, d_no_5_list = k_fold(X, y_5, 6)
d_wi_5_c_m, d_wi_5_list = k_fold(PCA_X, y_5, 7)
r_no_5_c_m, r_no_5_list = k_fold(X, y_5, 8)
r_wi_5_c_m, r_wi_5_list = k_fold(PCA_X, y_5, 9)
K_no_5_c_m, K_no_5_list = k_fold(X, y_5, 10)
K_wi_5_c_m, K_wi_5_list = k_fold(PCA_X, y_5, 11)

r_no_2_c_m_2, r_no_2_list_2 = k_fold(X, y_2, 12)
r_no_2_c_m_3, r_no_2_list_3 = k_fold(X, y_2, 13)

K_no_2_c_m_2, K_no_2_list_2 = k_fold(X, y_2, 14)
K_no_2_c_m_3, K_no_2_list_3 = k_fold(X, y_2, 15)

all_list = [d_no_2_list, d_wi_2_list, r_no_2_list, r_no_2_list_2, r_no_2_list_3, r_wi_2_list, K_no_2_list, K_no_2_list_2,
            K_no_2_list_3, K_wi_2_list, d_no_5_list, d_wi_5_list, r_no_5_list, r_wi_5_list, K_no_5_list, K_wi_5_list]
title_list =  ["Method", "Accuracy", "Recall(fail or V)", "Recall(pass or IV)",
                     "Recall(III)", "Recall(II)", "Recall(I)", "Precision(fail or V)",
                     "Precision(pass or IV)", "Precision(III)", "Precision(II)", 
                     "Precision(I)"] #把各個 column 名稱定好

table = pd.DataFrame(all_list, columns = title_list)
pd.set_option('display.width', 60)
pd.set_option('display.max_colwidth', 100)
table

C:\Users\55420\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Method,Accuracy,Recall(fail or V),Recall(pass or IV),Recall(III),Recall(II),Recall(I),Precision(fail or V),Precision(pass or IV),Precision(III),Precision(II),Precision(I)
0,Decision Tree without PCA (Binary Classification),0.89,0.82,0.92,/,/,/,0.83,0.91,/,/,/
1,Decision Tree with PCA (Binary Classification),0.81,0.70,0.86,/,/,/,0.72,0.86,/,/,/
2,Random Forest(65 trees) without PCA (Binary Classification),0.91,0.85,0.93,/,/,/,0.85,0.93,/,/,/
3,Random Forest(33 trees) without PCA (Binary Classification),0.89,0.77,0.94,/,/,/,0.85,0.90,/,/,/
4,Random Forest(17 trees) without PCA (Binary Classification),0.89,0.83,0.92,/,/,/,0.82,0.92,/,/,/
5,Random Forest(65 trees) with PCA (Binary Classification),0.82,0.56,0.94,/,/,/,0.82,0.81,/,/,/
6,KNN(K = 17) without PCA (Binary Classification),0.82,0.54,0.96,/,/,/,0.87,0.81,/,/,/
7,KNN(K = 11) without PCA (Binary Classification),0.82,0.53,0.96,/,/,/,0.85,0.81,/,/,/
8,KNN(K = 7) without PCA (Binary Classification),0.82,0.56,0.94,/,/,/,0.82,0.81,/,/,/
9,KNN(K = 17) with PCA (Binary Classification),0.82,0.52,0.96,/,/,/,0.86,0.81,/,/,/


> **Random Forest:** 我在 binary classification 中測試了 **65, 33, 17 trees**（選奇數才不會導致有平手的狀況），可以看到三者的正確率是差不多的，並沒有因為 tree 的數量增加而導致正確率上升。
  但 recall 和 precision 在 33 棵 tree 的時候都是最低的，代表我們的 tree 數量不能選得不大不小，而是應該選極端一點。
 
> **Difference between K-fold cross-validation and Random Forest:** Random Forest 是在 **training data** 中再去隨機取樣，並且也有可能會隨機取 feature；K-fold cross-validation 則是在**所有data中**(m 筆)取**連續**的 m/k 筆 data 當成 validation set，並且每個 feature 都會取到。 

> **KNN:** 我在 binary classification 中測試了 **K = 17, K = 11, K = 7** 這三種情況（選奇數才不會導致有平手的狀況），可以看到正確率隨著 K 下降而上升。  
而在 recall 方面，fail 的 recall **隨著 K 下降而上升**；相反地，pass 的 recall **隨著 K 上升而上升**。 <br>
precision 則和 recall 相反，fail 的 precision **隨著 K 上升而上升**，pass 的 precision **隨著 K 下降而上升**

In [235]:
c_m_list = [d_no_2_c_m, d_wi_2_c_m, r_no_2_c_m, r_wi_2_c_m, K_no_2_c_m, K_wi_2_c_m, 
             d_no_5_c_m, d_wi_5_c_m, r_no_5_c_m, r_wi_5_c_m, K_no_5_c_m, K_wi_5_c_m]

print(d_no_2_c_m)
#c_m_table = pd.DataFrame(c_m_list2, columns = ["Confusion Matrix"])

c_m_table = pt.PrettyTable()
c_m_table.field_names = ["Method", "Confusion Matrix"]

for c_m in c_m_list:
    c_m_table.add_row(c_m)
    c_m_table.add_row([" ", " "])

print(c_m_table)

['Decision Tree without PCA (Binary Classification)', array([[35.67,  7.67],
       [ 7.33, 81.  ]])]
+--------------------------------------------------------------+-----------------------------------+
|                            Method                            |          Confusion Matrix         |
+--------------------------------------------------------------+-----------------------------------+
|      Decision Tree without PCA (Binary Classification)       |           [[35.67  7.67]          |
|                                                              |           [ 7.33 81.  ]]          |
|                                                              |                                   |
|        Decision Tree with PCA (Binary Classification)        |           [[30.67 12.67]          |
|                                                              |           [12.   76.33]]          |
|                                                              |                          

## 7. Comparison & Conclusion

* 可以看到 binary classification 正確率遠高於 5-level classification，蠻合理的
* 在 binary classification 中，不管是 recall 還是 precision 都是 pass 表現較好，我推測是因為 pass 的人比 fail 的人多（請看下code）

In [236]:
zero, one = 0, 0

for y in y_2:
    if y == 0:
        zero += 1
    else:
        one += 1
        
print("pass: {}, fail: {}".format(one, zero))

pass: 265, fail: 130


* 在 5-level classification 中， recall 和 precision 幾乎都是 V 的表現最好，有可能是因為資料最多的關係（請看下code）。但應該不只是這個原因，因為其他的資料並沒有因為資料量較多而表現較好

In [237]:
I, II, III, IV, V = 0, 0, 0, 0, 0

for y in y_5:
    if y == 0:
        V += 1
    elif y == 1:
        IV += 1
    elif y == 2:
        III += 1
    elif y == 3:
        II += 1
    elif y == 4:
        I += 1
        
print("I: {}, II: {}, III: {}, IV: {}, V: {}".format(I, II, III, IV, V))

I: 40, II: 60, III: 62, IV: 103, V: 130


## 8. Questions 

### Decision Tree
* Show the prediction and reasoning of one arbitrary sample in the testing set.

In [238]:
pd.Series(PCA_X[0])

0    -0.865448
1    -1.818412
2     0.963255
3     0.341802
4     0.986816
5    -0.763197
6    -0.588321
7    -1.070194
8     0.361293
9     0.119632
10    0.267237
11   -0.237795
12   -0.972068
13    0.399961
14   -0.325599
15   -0.030632
16   -0.665288
17    0.076289
18   -0.628818
19   -0.014726
dtype: float64

* 使用這筆 data 當作 input，在 decision tree 上的 prediction 如下圖

### Random Forest
* Describe the difference between boosting and bagging.

1. 抽樣
    * **bagging**: 各個 data 被抽到的機率是相等的
    * **boosting**: 每次抽取的 data 是透過前一次的 model 預測結果來調整抽到的機率（錯誤率越高，機率越高）


2. 各個 decision tree 權重
    * **bagging**: 每個 decision tree 的權重都是相等的
    * **boosting**: decision tree 的正確率越高，權重越大


3. 平行運算
    * **bagging**: 可以平行運算（因為各個 decision tree 獨立）
    * **boosting**: 不可平行運算（因為每個 decision tree 都會根據前一個預測結果調整抽取 data 的機率）


4. 用途
    * **bagging**: 降低 variance
    * **boosting**: 降低 bias

### KNN
* Show the prediction and reasoning of one arbitrary sample in the testing set.

### KNN
* Bonus: pick 2 features, draw and describe the KNN decision boundaries.

### PCA
* In 5-Level classification, reduce the data dimension to 2 using PCA and draw a scatter plot. You have to colorize the data points based on their labels.